# HW3 Analysis

## Import and load data

In [1]:
import ml_pipeline as pp
import pandas as pd
import datetime as dt
import numpy as np
#from sklearn import metrics
file = './data/projects_2012_2013.csv'
df = pp.load_csv(file)
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

## Data transformations

### Helper functions for hw3 specific data

#### Convert date columns to datetime

In [2]:
df.date_posted = pp.col_datetime(df, 'date_posted')

In [3]:
df.datefullyfunded = pp.col_datetime(df,'datefullyfunded')

#### Create labels: 1 if not fully funded in 60 days, 0 if fully funded in 60 days

In [4]:
df = pp.create_label(df, pred_time=60)

In [5]:
df.head()

,projectid,teacher_acctid,schoolid,school_ncesid,school_latitude,school_longitude,school_city,school_state,school_metro,school_district,...,secondary_focus_area,resource_type,poverty_level,grade_level,total_price_including_optional_support,students_reached,eligible_double_your_impact_match,date_posted,datefullyfunded,label
0,00001ccc0e81598c4bd86bacb94d7acb,96963218e74e10c3764a5cfb153e6fea,9f3f9f2c2da7edda5648ccd10554ed8c,1.709930e+11,41.807654,-87.673257,Chicago,IL,urban,Pershing Elem Network,...,Music & The Arts,Supplies,highest poverty,Grades PreK-2,1498.61,31.0,f,2013-04-14,2013-05-02,0
1,0000fa3aa8f6649abab23615b546016d,2a578595fe351e7fce057e048c409b18,3432ed3d4466fac2f2ead83ab354e333,6.409801e+10,34.296596,-119.296596,Ventura,CA,urban,Ventura Unif School District,...,Literacy & Language,Books,highest poverty,Grades 3-5,282.47,28.0,t,2012-04-07,2012-04-18,0
2,000134f07d4b30140d63262c871748ff,26bd60377bdbffb53a644a16c5308e82,dc8dcb501c3b2bb0b10e9c6ee2cd8afd,6.227100e+10,34.078625,-118.257834,Los Angeles,CA,urban,Los Angeles Unif Sch Dist,...,History & Civics,Technology,high poverty,Grades 3-5,1012.38,56.0,f,2012-01-30,2012-04-15,1
3,0001f2d0b3827bba67cdbeaa248b832d,15d900805d9d716c051c671827109f45,8bea7e8c6e4279fca6276128db89292e,3.600090e+11,40.687286,-73.988217,Brooklyn,NY,urban,New York City Dept Of Ed,...,NaN,Books,high poverty,Grades PreK-2,175.33,23.0,f,2012-10-11,2012-12-05,0
4,0004536db996ba697ca72c9e058bfe69,400f8b82bb0143f6a40b217a517fe311,fbdefab6fe41e12c55886c610c110753,3.606870e+11,40.793018,-73.205635,Central Islip,NY,suburban,Central Islip Union Free SD,...,Literacy & Language,Technology,high poverty,Grades PreK-2,3591.11,150.0,f,2013-01-08,2013-03-25,1


## Selecting features and cleaning

In [6]:
feature_cols=['school_metro','school_charter', 'school_magnet', 'primary_focus_subject', 'primary_focus_area', 'resource_type', 'poverty_level', 'grade_level', 'total_price_including_optional_support', 'students_reached', 'eligible_double_your_impact_match', 'date_posted', 'label']
sel = df[feature_cols].copy()
sel.head()

,school_metro,school_charter,school_magnet,primary_focus_subject,primary_focus_area,resource_type,poverty_level,grade_level,total_price_including_optional_support,students_reached,eligible_double_your_impact_match,date_posted,label
0,urban,f,f,Mathematics,Math & Science,Supplies,highest poverty,Grades PreK-2,1498.61,31.0,f,2013-04-14,0
1,urban,f,f,Civics & Government,History & Civics,Books,highest poverty,Grades 3-5,282.47,28.0,t,2012-04-07,0
2,urban,f,f,Literacy,Literacy & Language,Technology,high poverty,Grades 3-5,1012.38,56.0,f,2012-01-30,1
3,urban,f,t,Literacy,Literacy & Language,Books,high poverty,Grades PreK-2,175.33,23.0,f,2012-10-11,0
4,suburban,f,f,Literacy,Literacy & Language,Technology,high poverty,Grades PreK-2,3591.11,150.0,f,2013-01-08,1


#### Identify feature columns with null values

In [7]:
for x in pp.na_col(df):
    if x in feature_cols:
        print(x)

school_metro
primary_focus_subject
primary_focus_area
resource_type
grade_level
students_reached


#### Impute missing categorical variables with the most frequent, which is a common way to handle missing categorical data without more information. 


In [8]:
cat_cols = ['school_metro','primary_focus_subject','primary_focus_area','resource_type','grade_level']
for x in cat_cols:
    sel = pp.na_fill_col(sel, x , pp.most_freq)

#### Impute missing numerical variable (students_reached) with the median value because there are outliers affecting the mean. 

In [9]:
sel.students_reached.quantile([0.1, 0.25, 0.5, 0.75, 0.9,0.98,1])

0.10       18.0
0.25       23.0
0.50       30.0
0.75      100.0
0.90      200.0
0.98      700.0
1.00    12143.0
Name: students_reached, dtype: float64

In [10]:
sel = pp.na_fill_col(sel, 'students_reached', np.nanmedian)

#### Check that there are no more missing values in feature columns

In [11]:
for x in pp.na_col(sel):
    if x in feature_cols:
        print(x)

#### Discretize numeric features and then get all dummy variables.

In [12]:
# discretize numeric features
bucketdict= {'total_price_including_optional_support': 4, 'students_reached':4}
df_discr = pp.feat_mult_disc(sel, bucketdict, qt=True)

df_discr.total_price_including_optional_support_binned.unique()
df_discr.students_reached_binned.unique()

[(30.0, 100.0], (23.0, 30.0], (0.999, 23.0], (100.0, 12143.0]]
Categories (4, interval[float64]): [(0.999, 23.0] < (23.0, 30.0] < (30.0, 100.0] < (100.0, 12143.0]]

In [13]:
col_to_binary = list(df_discr.columns)
col_to_binary.remove('label')
col_to_binary.remove('date_posted')

In [14]:
col_to_binary

['school_metro',
 'school_charter',
 'school_magnet',
 'primary_focus_subject',
 'primary_focus_area',
 'resource_type',
 'poverty_level',
 'grade_level',
 'eligible_double_your_impact_match',
 'total_price_including_optional_support_binned',
 'students_reached_binned']

In [15]:
# turn variables into dummies
df_final = pp.feat_binary(df_discr, col_to_binary)
df_final.head()

,date_posted,label,school_metro_rural,school_metro_suburban,school_metro_urban,school_charter_f,school_charter_t,school_magnet_f,school_magnet_t,primary_focus_subject_Applied Sciences,...,eligible_double_your_impact_match_f,eligible_double_your_impact_match_t,"total_price_including_optional_support_binned_(91.999, 345.81]","total_price_including_optional_support_binned_(345.81, 510.5]","total_price_including_optional_support_binned_(510.5, 752.96]","total_price_including_optional_support_binned_(752.96, 164382.84]","students_reached_binned_(0.999, 23.0]","students_reached_binned_(23.0, 30.0]","students_reached_binned_(30.0, 100.0]","students_reached_binned_(100.0, 12143.0]"
0,2013-04-14,0,0,0,1,1,0,1,0,0,...,1,0,0,0,0,1,0,0,1,0
1,2012-04-07,0,0,0,1,1,0,1,0,0,...,0,1,1,0,0,0,0,1,0,0
2,2012-01-30,1,0,0,1,1,0,1,0,0,...,1,0,0,0,0,1,0,0,1,0
3,2012-10-11,0,0,0,1,1,0,0,1,0,...,1,0,1,0,0,0,1,0,0,0
4,2013-01-08,1,0,1,0,1,0,1,0,0,...,1,0,0,0,0,1,0,0,0,1


In [18]:
df_final.to_csv('./data/cleaned.csv', index=False)

## Run variations of models: 
### Decision trees, KNN, Logistic Regression, Linear SVM, Random forests, Bagging, Boosting

In [16]:
windows = [dt.datetime(2012,1,1), dt.datetime(2012,7,1), dt.datetime(2013,1,1), dt.datetime(2013,7,1), dt.datetime(2014,1,1)]
pred_time = 60 #days
label_col = 'label'
split_col = 'date_posted'
feature_cols= list(df_final.columns)
feature_cols.remove('label')
feature_cols.remove('date_posted')
seed=12345

In [17]:
models = [
    {'type': 'Dtree', 'clf': pp.dtree_score, 'criteria': ['entropy', 'gini'], 'depth': [10,20,30],'min_leaf': [100, 300,500], 'seed': seed},
    {'type': 'LR', 'clf': pp.lr_score, 'p': ['l1','l2'], 'c': [0.1, 1.0, 10.0, 100.0], 'solver': ['liblinear'], 'seed': seed},
    {'type': 'SVM', 'clf': pp.linsvc_score, 'p': ['l2'], 'c': [0.1, 1.0, 10.0, 100.0], 'seed': seed},
    {'type': 'Bagging_dtree', 'clf': pp.bagging_score, 'n': [10, 50, 100], 'base':[None], 'seed':seed},
    {'type': 'ADABoost_dtree', 'clf': pp.adaboost_score, 'n': [10, 50, 100], 'base':[None], 'seed':seed},
    {'type': 'Random Forest', 'clf': pp.rforest_score, 'n': [10, 50, 100], 'criterion': ['entropy', 'gini'], 'seed': seed},
    {'type': 'KNN', 'clf': pp.knn_score, 'n': [5], 'weights': ['uniform','distance'], 'distance_metric':['minkowski'],'p': [1,2]}
]

#models = [{'type': 'Random Forest', 'clf': rforest_score, 'n': [10, 50, 100], 'criterion': ['entropy', 'gini'], 'seed': seed}]
thresholds = [1, 2, 5, 10, 20,30, 50]


In [18]:
resdf = pp.run_models(models, thresholds, windows, df_final, feature_cols, label_col, split_col, pred_time, pred_unit = 'day', filename = './data/finalrun.csv')
resdf.head()

model: Dtree, run: 1
criteria: entropy, depth: 10, min_leaf: 100, seed: 12345
criteria: entropy, depth: 10, min_leaf: 300, seed: 12345
criteria: entropy, depth: 10, min_leaf: 500, seed: 12345
criteria: entropy, depth: 20, min_leaf: 100, seed: 12345
criteria: entropy, depth: 20, min_leaf: 300, seed: 12345
criteria: entropy, depth: 20, min_leaf: 500, seed: 12345
criteria: entropy, depth: 30, min_leaf: 100, seed: 12345
criteria: entropy, depth: 30, min_leaf: 300, seed: 12345
criteria: entropy, depth: 30, min_leaf: 500, seed: 12345
criteria: gini, depth: 10, min_leaf: 100, seed: 12345
criteria: gini, depth: 10, min_leaf: 300, seed: 12345
criteria: gini, depth: 10, min_leaf: 500, seed: 12345
criteria: gini, depth: 20, min_leaf: 100, seed: 12345
criteria: gini, depth: 20, min_leaf: 300, seed: 12345
criteria: gini, depth: 20, min_leaf: 500, seed: 12345
criteria: gini, depth: 30, min_leaf: 100, seed: 12345
criteria: gini, depth: 30, min_leaf: 300, seed: 12345
criteria: gini, depth: 30, min_lea

,type,details,baseline,threshold_pct,precision,recall,auc,train_set_num,train_start,test_start
0,Dtree,"criteria: entropy, depth: 10, min_leaf: 100, s...",0.743083,1,0.948485,0.012780,0.505386,1,2012-01-01,2012-07-01
1,Dtree,"criteria: entropy, depth: 10, min_leaf: 100, s...",0.743083,2,0.849772,0.022865,0.505587,1,2012-01-01,2012-07-01
2,Dtree,"criteria: entropy, depth: 10, min_leaf: 100, s...",0.743083,5,0.778519,0.052384,0.504641,1,2012-01-01,2012-07-01
3,Dtree,"criteria: entropy, depth: 10, min_leaf: 100, s...",0.743083,10,0.755158,0.101625,0.503163,1,2012-01-01,2012-07-01
4,Dtree,"criteria: entropy, depth: 10, min_leaf: 100, s...",0.743083,20,0.749242,0.201658,0.503226,1,2012-01-01,2012-07-01


#### Testing code

In [ ]:
#### Use temporal validation to split into training/test sets



x_train1,y_train1,x_test1,y_test1 = pp.single_train_test_set(df_final, 
                                                          feature_cols, 
                                                          label_col, 
                                                          split_col, 
                                                          windows[0],
                                                          windows[1], 
                                                          windows[2], 
                                                          pred_time=60)

x_train2, y_train2, x_test2, y_test2 = pp.single_train_test_set(df_final, 
                                                          feature_cols, 
                                                          label_col, 
                                                          split_col, 
                                                          windows[0],
                                                          windows[2], 
                                                          windows[3], 
                                                          pred_time=60)

x_train3, y_train3, x_test3, y_test3 = pp.single_train_test_set(df_final, 
                                                          feature_cols, 
                                                          label_col, 
                                                          split_col, 
                                                          windows[0],
                                                          windows[3], 
                                                          windows[4], 
                                                          pred_time=60)

In [ ]:
import matplotlib.pyplot as plt
x = lr_score(x_train1, y_train1, x_test1, p = 'l1', c = 10.0, solver = 'liblinear', seed=12345)
precision_at_threshold(y_test1, x, 0)
plot_pct_pop(y_test1, x)


In [ ]:
def plot_pct_pop(y_test, pred_scores):
    '''
    This function plots precision and recall on two axes with percent of population as the x-axis.
    
    y_test: test set labels
    pred_scores: predicted scores
    
    return: None
    '''
    pct_pop = np.array([1, 10, 20, 30, 40, 50, 60, 70, 80, 90, 99])
    prec = []
    rec = []
    for each in pct_pop:
        #quantile = 100 - each
        #t = np.percentile(pred_scores, quantile)
        a, p, r, f1, auc = all_metrics(y_test, pred_scores, each)
        #p = precision_at_threshold(y_test, pred_scores, t)
        #r = recall_at_threshold(y_test, pred_scores, t)
        prec.append(p)
        rec.append(r)
    
    #pct_pop = 1-thresholds
    fig, ax1 = plt.subplots()

    color = 'tab:blue'
    ax1.set_xlabel('percent of population')
    ax1.set_ylabel('precision', color=color)
    ax1.plot(pct_pop, prec, color=color)
    ax1.tick_params(axis='y', labelcolor=color)
    plt.yticks(np.arange(0, 1.2, step=0.2))
    ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis

    color = 'tab:red'
    ax2.set_ylabel('recall', color=color)  # we already handled the x-label with ax1
    ax2.plot(pct_pop, rec, color=color)
    ax2.tick_params(axis='y', labelcolor=color)
    plt.yticks(np.arange(0, 1.2, step=0.2))
    fig.tight_layout()  # otherwise the right y-label is slightly clipped
    plt.show()

In [4]:
resdf = pp.load_csv('./data/finalrun.csv')

## Comparing model results

#### Identify models that do better on precision, recall, and AUC by threshold and time period (top two for each)

In [34]:
best_prec = resdf.sort_values('precision', ascending=False)
best_prec.groupby(['train_set_num', 'threshold_pct']).nth([0,1])

type  \
train_set_num threshold_pct                   
1             1                       Dtree   
              1                       Dtree   
              2                       Dtree   
              2                       Dtree   
              5                       Dtree   
              5                          LR   
              10                      Dtree   
              10                      Dtree   
              20                        SVM   
              20                      Dtree   
              30              Random Forest   
              30                      Dtree   
              50                        SVM   
              50                         LR   
2             1                          LR   
              1              ADABoost_dtree   
              2                       Dtree   
              2                       Dtree   
              5                          LR   
              5                       Dtree   
              10             ADABoost_dtree   
              10                         LR   
              20             ADABoost_dtree   
              20                      Dtree   
              30                         LR   
              30                        SVM   
              50                        SVM   
              50                         LR   
3             1                       Dtree   
              1                       Dtree   
              2                       Dtree   
              2                       Dtree   
              5                       Dtree   
              5                       Dtree   
              10                         LR   
              10             ADABoost_dtree   
              20                      Dtree   
              20                      Dtree   
              30                      Dtree   
              30                      Dtree   
              50             ADABoost_dtree   
              50                      Dtree   

                                                                       details  \
train_set_num threshold_pct                                                      
1             1              criteria: entropy, depth: 10, min_leaf: 100, s...   
              1              criteria: gini, depth: 10, min_leaf: 100, seed...   
              2              criteria: entropy, depth: 10, min_leaf: 100, s...   
              2              criteria: entropy, depth: 30, min_leaf: 100, s...   
              5              criteria: entropy, depth: 10, min_leaf: 100, s...   
              5              penalty: l1, c: 100.0, solver: liblinear, seed...   
              10             criteria: entropy, depth: 30, min_leaf: 100, s...   
              10             criteria: entropy, depth: 20, min_leaf: 100, s...   
              20                            penalty: l2, c: 100.0, seed: 12345   
              20             criteria: entropy, depth: 10, min_leaf: 100, s...   
              30                                    n: 100, criterion: entropy   
              30             criteria: gini, depth: 10, min_leaf: 500, seed...   
              50                            penalty: l2, c: 100.0, seed: 12345   
              50             penalty: l1, c: 0.1, solver: liblinear, seed: ...   
2             1              penalty: l2, c: 0.1, solver: liblinear, seed: ...   
              1                                              n: 10, base: None   
              2              criteria: gini, depth: 10, min_leaf: 300, seed...   
              2              criteria: entropy, depth: 10, min_leaf: 300, s...   
              5              penalty: l2, c: 0.1, solver: liblinear, seed: ...   
              5              criteria: entropy, depth: 30, min_leaf: 300, s...   
              10                                             n: 10, base: None   
              10             penalty: l2, c: 0.1, solver: liblinear, seed: ...   
  

In [32]:
resdf[(resdf['train_set_num'] == 2)&(resdf['threshold_pct'] == 5)].sort_values('recall', ascending=False)

,type,details,baseline,threshold_pct,precision,recall,auc,train_set_num,train_start,test_start
478,LR,"penalty: l2, c: 0.1, solver: liblinear, seed: ...",0.684111,5,0.737327,0.053872,0.506154,2,2012-01-01,2013-01-01
436,Dtree,"criteria: gini, depth: 30, min_leaf: 300, seed...",0.684111,5,0.734562,0.053670,0.505835,2,2012-01-01,2013-01-01
415,Dtree,"criteria: gini, depth: 20, min_leaf: 300, seed...",0.684111,5,0.734562,0.053670,0.505835,2,2012-01-01,2013-01-01
352,Dtree,"criteria: entropy, depth: 20, min_leaf: 300, s...",0.684111,5,0.734562,0.053670,0.505835,2,2012-01-01,2013-01-01
373,Dtree,"criteria: entropy, depth: 30, min_leaf: 300, s...",0.684111,5,0.734562,0.053670,0.505835,2,2012-01-01,2013-01-01
555,ADABoost_dtree,"n: 10, base: None",0.684111,5,0.733641,0.053603,0.505728,2,2012-01-01,2013-01-01
520,SVM,"penalty: l2, c: 10.0, seed: 12345",0.684111,5,0.732719,0.053535,0.505621,2,2012-01-01,2013-01-01
513,SVM,"penalty: l2, c: 1.0, seed: 12345",0.684111,5,0.732719,0.053535,0.505621,2,2012-01-01,2013-01-01
506,SVM,"penalty: l2, c: 0.1, seed: 12345",0.684111,5,0.732719,0.053535,0.505621,2,2012-01-01,2013-01-01
527,SVM,"penalty: l2, c: 100.0, seed: 12345",0.684111,5,0.731797,0.053468,0.505515,2,2012-01-01,2013-01-01


In [35]:
resdf.sort_values('auc', ascending=False)

,type,details,baseline,threshold_pct,precision,recall,auc,train_set_num,train_start,test_start
737,Dtree,"criteria: gini, depth: 20, min_leaf: 300, seed...",0.715353,5,0.772645,0.053996,0.507033,3,2012-01-01,2013-07-01
758,Dtree,"criteria: gini, depth: 30, min_leaf: 300, seed...",0.715353,5,0.772645,0.053996,0.507033,3,2012-01-01,2013-07-01
695,Dtree,"criteria: entropy, depth: 30, min_leaf: 300, s...",0.715353,5,0.771739,0.053933,0.506922,3,2012-01-01,2013-07-01
646,Dtree,"criteria: entropy, depth: 10, min_leaf: 100, s...",0.715353,5,0.771739,0.053933,0.506922,3,2012-01-01,2013-07-01
744,Dtree,"criteria: gini, depth: 20, min_leaf: 500, seed...",0.715353,5,0.771739,0.053933,0.506922,3,2012-01-01,2013-07-01
653,Dtree,"criteria: entropy, depth: 10, min_leaf: 300, s...",0.715353,5,0.771739,0.053933,0.506922,3,2012-01-01,2013-07-01
765,Dtree,"criteria: gini, depth: 30, min_leaf: 500, seed...",0.715353,5,0.771739,0.053933,0.506922,3,2012-01-01,2013-07-01
674,Dtree,"criteria: entropy, depth: 20, min_leaf: 300, s...",0.715353,5,0.771739,0.053933,0.506922,3,2012-01-01,2013-07-01
716,Dtree,"criteria: gini, depth: 10, min_leaf: 300, seed...",0.715353,5,0.771286,0.053901,0.506866,3,2012-01-01,2013-07-01
884,ADABoost_dtree,"n: 50, base: None",0.715353,5,0.770833,0.053869,0.506811,3,2012-01-01,2013-07-01


In [22]:
resdf.groupby(['train_set_num', 'threshold_pct']).agg({'precision' : 'max', 'recall' : 'max', 'auc': 'max'})

precision    recall       auc
train_set_num threshold_pct                               
1             1               0.948485  0.012780  0.505386
              2               0.849772  0.022865  0.505587
              5               0.778519  0.052384  0.504641
              10              0.755461  0.101666  0.503242
              20              0.749697  0.201780  0.503464
              30              0.746258  0.301282  0.502495
              50              0.741869  0.499183  0.498411
2             1               0.963134  0.014074  0.506454
              2               0.817972  0.023906  0.506192
              5               0.737327  0.053872  0.506154
              10              0.700599  0.102424  0.503815
              20              0.684174  0.200000  0.500029
              30              0.681050  0.298653  0.497876
              50              0.681868  0.498384  0.497405
3             1               0.966063  0.013515  0.506161
              2               0.841450  0.023516  0.506190
              5               0.772645  0.053996  0.507033
              10              0.741227  0.103624  0.506354
              20              0.727499  0.203387  0.505965
              30              0.720981  0.302358  0.504146
              50              0.720069  0.503307  0.505790

#### Examine how results change over time

#### Choose model for the 5% threshold (target percent fo population)